# HSR Mujoco Example

HSR in Mujoco simulator.

### Dependencies:

1. [Giskard branch: mujoco_actions_devel](https://github.com/SemRoCo/giskardpy/tree/37b4048c3787c1f5f0370ca4e8ee10754eaa12c9)

1. [mujoco_sim](https://github.com/HoangGiang93/mujoco_sim)

1. [HSR description files](https://github.com/maltehue/mujoco_robots)

## 1. Launch Robot and envrionment

# What is the bootstrapping approach for the task learning?
In the context of teaching, the bootstrapping approach often involves starting with basic knowledge or skills and then iteratively improving or expanding upon them through additional learning experiences, in this case by interacting with a human mentor. This approach allows learners to gradually acquire more complex knowledge or abilities by building on what they already know. Bootstrapping is a common strategy in education and learning systems, helping individuals incrementally develop their expertise.

For the learning of the pouring task, a task-learning strategy has been developed by leveraging primitive actions that the PR2 robot can execute using GISKARD's primitive motion skills. This strategy involves breaking down the pouring task into already existing skills that the robot knows how to execute. By utilizing these primitive skills, the robot can gradually learn and perform the complete pouring task. When the instruction to save the learning plan is given, the robot can then incorporate this complex plan into its existing repertoire of skills. This method allows the robot to accumulate and store knowledge about various tasks and their constraints, contributing to its overall capabilities.

In [1]:
import os
import psutil
import subprocess
import rospy
from ipywidgets import Button, Layout, GridBox, VBox, FloatSlider, ToggleButtons, Box
from IPython.display import display, Markdown, IFrame
from sidecar import Sidecar

# Directory of the ROS launch files
LAUNCH_FILE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../launch"))
# To display web UI on the left
SIDECAR = None
# To manage the roslaunch process in the background
LAUNCH_PROCESS = None

# UI widgets variables
UI_CONFIG = {
    'robots': [
        {
            'name': 'HSR with a cup',
            'value': '$(find hsr_mujoco)/model/hsrb4s.xml',
        }
    ],
    'worlds': [
        {
            'name': 'world_particle_container',
            'value': '$(find hsr_mujoco)/model/world_two_cups.xml',
        }
    ]
}
SELECTED_ROBOT = UI_CONFIG['robots'][0]['value']
SELECTED_WORLD = UI_CONFIG['worlds'][0]['value']

try:
    DISPLAY_NUM = os.environ['DISPLAY']
except KeyError:
    DISPLAY_NUM = None

# Popup the xpra desktop on the left panel
def open_xpra():
    global SIDECAR
    if SIDECAR is not None:
        SIDECAR.close()
    SIDECAR = Sidecar(title='XRPA', anchor='right')
    try:
        JUPYTERHUB_USER = '/user/' + os.environ['JUPYTERHUB_USER']
    except KeyError:
        JUPYTERHUB_USER = ''
    xpra_url = f"{JUPYTERHUB_USER}/xprahtml5/index.html"
    print(xpra_url)
    with SIDECAR:
        display(IFrame(src=xpra_url, width='100%', height='100%'))

# Execute the roslaunch command
def launch_simulation():
    global LAUNCH_PROCESS
    if LAUNCH_PROCESS is not None:
        LAUNCH_PROCESS.terminate()
        LAUNCH_PROCESS.wait()
    if DISPLAY_NUM == ':100' or True:
        open_xpra()
    launchfile = os.path.join(LAUNCH_FILE_DIR, "hsrb4s_velocity_two_cups.launch")
    command = [
        'roslaunch', 
        launchfile,
        'gui:=false',
        'web:=false',
        f"mujoco_robot:={SELECTED_ROBOT}",
        f"mujoco_world:={SELECTED_WORLD}"
    ]
    LAUNCH_PROCESS = psutil.Popen(command,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL)
    print(f"""
        Executing command: 
        {' '.join(command)}
        in the background, the output will be hidden. 
        To check the output please execute it in a Terminial!
    """)
    launch_giskard()
    
# Execute the roslaunch command
def launch_giskard():
    global LAUNCH_PROCESS
    if LAUNCH_PROCESS is not None:
        LAUNCH_PROCESS.terminate()
        LAUNCH_PROCESS.wait()
    if DISPLAY_NUM == ':100' or True:
        open_xpra()
    launchfile = os.path.join(LAUNCH_FILE_DIR, "giskardpy_hsr_mujoco.launch")
    command = [
        'roslaunch', 
        launchfile,
        'gui:=false',
        'web:=false',
        f"mujoco_robot:={SELECTED_ROBOT}",
        f"mujoco_world:={SELECTED_WORLD}"
    ]
    LAUNCH_PROCESS = psutil.Popen(command,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL)
    print(f"""
        Executing command: 
        {' '.join(command)}
        in the background, the output will be hidden. 
        To check the output please execute it in a Terminial!
    """)

# Create UI widgets
def select_robot(index):
    global SELECTED_ROBOT
    SELECTED_ROBOT = UI_CONFIG['robots'][index]['value']

def select_world(index):
    global SELECTED_WORLD
    SELECTED_WORLD = UI_CONFIG['worlds'][index]['value']

def create_widgets(config):
    widgets = []
    robot_select = ToggleButtons(
        options=[i['name'] for i in config['robots']],
        description='robots',
        tooltips=[i['value'] for i in config['robots']],
    )
    robot_select.observe(lambda v: select_robot(v['new']), names="index")

    world_select = ToggleButtons(
        options=[i['name'] for i in config['worlds']],
        description='worlds',
        tooltips=[i['value'] for i in config['worlds']],
    )
    world_select.observe(lambda v: select_world(v['new']), names="index")

    Launch_btn = Button(
        description='Launch Simulation',
        layout=Layout(width='auto'),
        button_style='success'
    )
    Launch_btn.on_click(lambda b: launch_simulation())
    widgets.append(robot_select)
    widgets.append(world_select)
    widgets.append(Launch_btn)
    return VBox(widgets)

# Display widgets
display(create_widgets(UI_CONFIG))

## 2. Control the robot with the Giskard Python Interface

In [2]:
import rospy
import random
from giskardpy.python_interface.python_interface import GiskardWrapper
from giskardpy.goals.joint_goals import JointPositionList
from giskardpy.monitors.joint_monitors import JointGoalReached
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, QuaternionStamped
from grasp_and_pour_methods import openGripper, align_to, closeGripper, tilt, grasp

# Before running this script make sure to start a giskard instance using 'roslaunch giskardpy giskardpy_hsr_mujoco.launch'
# And before that the mujoco simulation has to be running
rospy.init_node('graspAndPour')
gis = GiskardWrapper()

# Define some parameters used in the movement function
# The endeffector link of the robot
robot_eeff = 'hand_palm_link'

# Axis of the eeff that should be upright
upright_axis = Vector3Stamped()
upright_axis.header.frame_id = robot_eeff
upright_axis.vector.x = 1

# A second axis of the eeff. Can be aligned to the x-axis of goal objects
second_axis = Vector3Stamped()
second_axis.header.frame_id = robot_eeff
second_axis.vector.z = 1

# Grasp the free cup from front side 
grasp(gis, 'free_cup', robot_eeff, 'front', upright_axis, second_axis)

# Here the grasped cup is added to the kinematic model of the robot.
# It is done to use the frame of the cup as a controlled frame.
# This can be skipped if the 'hand_palm_link' should be used as a controlled frame after grasping.
# First define the current pose of the grasped cup
cup_pose = PoseStamped()
cup_pose.header.frame_id = 'free_cup'
cup_pose.pose.position = Point(0, 0, 0)
cup_pose.pose.orientation.w = 1
# Add the cup to the robot model name 'grasped_cup' and the known dimensions
gis.world.add_box('grasped_cup', (0.07, 0.07, 0.18), pose=cup_pose, parent_link=robot_eeff)
# Now update the robot_eeff reference to use the grasped cup
robot_eeff = 'grasped_cup'

# This aligns the control frame to the left of the object frame in a distance of 0.13m.
# Additionally, the control frame is 0.2m higher than the object frame. The second_distance paramter can be used to
# to set an offset in the remaining dimension, here the x-axis of the object frame
align_to(gis, 'left', axis_align_to_z=upright_axis, object_frame='free_cup2', control_frame=robot_eeff,
         axis_align_to_x=second_axis, distance=0.13, height_offset=0.2, second_distance=0.0)

# Prepare tilting be defining a tilt axis
rotation_axis = Vector3Stamped()
rotation_axis.header.frame_id = robot_eeff
rotation_axis.vector.z = 1

# Tilt the controlled_frame by angle around the rotation axis with a maximum velocity of velocity.
tilt(gis, angle=1.7, velocity=1.0, rotation_axis=rotation_axis, controlled_frame='hand_palm_link')
